# Analyse de la Pyramide des Âges des Élus en France


L'analyse présentée dans ce notebook vise à explorer la répartition des élus en France selon leur âge et leur sexe. Les données utilisées proviennent du Répertoire National des Élus, disponible sur le site data.gouv.fr à l'adresse https://static.data.gouv.fr/resources/repertoire-national-des-elus-1/20250613-142903/elus-maires-mai.csv. Ce fichier contient des informations détaillées sur les élus, notamment leurs données personnelles, professionnelles et les détails de leurs mandats.


L'objectif de cette analyse est de générer une pyramide des âges par sexe et par tranche de 5 ans pour les élus. Cela permettra de visualiser la distribution des élus selon leur âge et leur sexe, offrant ainsi des insights précieux sur la représentation des différents groupes démographiques dans la sphère politique française.


## Méthodologie


La méthodologie utilisée dans cette analyse implique plusieurs étapes clés. Tout d'abord, les données sont chargées et nettoyées pour préparer l'analyse. Ensuite, l'âge de chaque élu est calculé en fonction de sa date de naissance, et les élus sont regroupés par tranche d'âge de 5 ans. Les données sont alors agrégées pour compter le nombre d'élus par tranche d'âge et par sexe. Enfin, les résultats sont visualisés sous forme de pyramide des âges à l'aide d'une bibliothèque de visualisation de données. Cette approche permet d'obtenir une représentation claire et intuitive de la distribution des élus selon leur âge et leur sexe.

## 🔧 Configuration

In [1]:
# Installation et imports
import duckdb as ddb
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go

## 🦆 Chargement du dataset avec Duckdb

In [1]:
# Fonction de chargement complète (basée sur load_file_from_url_lite)
def load_file_from_url_lite(url_dataset="", loader="read_csv_auto", options="", nom_table="loaded_dataset", safe_mode=False):
    ddb.execute("install spatial")
    ddb.execute("load spatial")
    ddb.execute("INSTALL h3 FROM community")
    ddb.execute("LOAD h3")
    ddb.execute("install webbed from community;")
    ddb.execute("load webbed")
    ddb.execute("set force_download=True")
    ddb.execute(f"drop table if exists {nom_table}")   
    
    # Détection automatique du type de fichier
    if 'csv' in url_dataset: 
        loader = "read_csv_auto"
    elif 'tsv' in url_dataset: 
        loader = "read_csv_auto"
    elif 'txt' in url_dataset: 
        loader = "read_csv_auto"
    elif 'parquet' in url_dataset: 
        loader = "read_parquet"
    elif 'json' in url_dataset: 
        loader = "read_json_auto"
    elif 'xls' in url_dataset or 'xlsx' in url_dataset: 
        loader = "st_read"
    elif 'shp' in url_dataset: 
        loader = "st_read"
    elif 'geojson' in url_dataset: 
        loader = "st_read"
    elif 'xml' in url_dataset: 
        loader = "read_xml"
    elif 'html' in url_dataset: 
        loader = "read_html"
    else: 
        raise ValueError(f"Type de fichier non supporté pour {url_dataset}")
    
    if options=="": 
        options = "" 
    if 'csv' in url_dataset and safe_mode==True: 
        options = ", all_varchar=1" 
    if nom_table=="": 
        nom_table = "loaded_dataset"
    
    try:
        status = ddb.sql(f"""
            create or replace table {nom_table} as select *
            from
            {loader}("{url_dataset}" {options})
        """)
        return status
    except Exception as e:
        return f"Erreur au chargement du fichier : {str(e)}"

def run_query(sql):
    return ddb.sql(sql.replace("`"," ")).to_df()

# Chargement des données
load_file_from_url_lite("https://static.data.gouv.fr/resources/repertoire-national-des-elus-1/20250613-142903/elus-maires-mai.csv", safe_mode=True)
print("✅ Données chargées avec succès")

## 🔍 Analyse SQL

Cette requête utilise des techniques SQL pour extraire et transformer les données de manière efficace.

In [2]:
# Exécution de la requête
df = run_query(""" WITH age_calcul AS (
  SELECT 
    "Code sexe" AS sexe,
    CAST(SUBSTRING("Date de naissance", 7, 4) AS INTEGER) AS annee_naissance,
    FLOOR((2023 - CAST(SUBSTRING("Date de naissance", 7, 4) AS INTEGER)) / 5) * 5 AS age_tranche
  FROM loaded_dataset
),
pyramide AS (
  SELECT 
    sexe,
    age_tranche,
    COUNT(*) AS nombre_elus
  FROM age_calcul
  GROUP BY ALL
)
SELECT 
  sexe,
  age_tranche,
  nombre_elus
FROM pyramide
ORDER BY sexe, age_tranche """)
print(f"Résultats : {len(df)} lignes")
df.head()

## 📈 Visualisation

La bibliothèque principale utilisée est Plotly, qui permet de créer des visualisations interactives et personnalisables. Cette technologie est adaptée pour représenter une pyramide des âges sous forme de diagramme à barres horizontales superposées, facilitant la comparaison entre les sexes. L'utilisation de Plotly permet une représentation dynamique et facile à lire.

In [3]:
import pandas as pd
import duckdb as ddb
import pandas as pd
import plotly.graph_objects as go

# Tranches de 5 ans, ordre croissant
df["age_min"] = (df["age_tranche"] // 5) * 5
grp = (
    df.groupby(["age_min", "sexe"], as_index=False)["nombre_elus"]
      .sum()
      .sort_values(["age_min", "sexe"])
)

labels = [f'{int(a)}-{int(a+4)} ans' for a in grp["age_min"].unique()]
f_values = grp[grp["sexe"] == "F"].set_index("age_min")["nombre_elus"].reindex(grp["age_min"].unique(), fill_value=0).values
m_values = grp[grp["sexe"] == "M"].set_index("age_min")["nombre_elus"].reindex(grp["age_min"].unique(), fill_value=0).values

dataviz = go.Figure()
dataviz.add_trace(go.Bar(
    y=labels,
    x=-f_values,
    name="Femmes",
    orientation="h",
    hovertemplate="%{x}<extra>Femmes</extra>",
    marker_color="#ff8599"
))
dataviz.add_trace(go.Bar(
    y=labels,
    x=m_values,
    name="Hommes",
    orientation="h",
    hovertemplate="%{x}<extra>Hommes</extra>",
    marker_color="#6ca0dc"
))

dataviz.update_layout(
    title="Pyramide des âges des élus par sexe - Politique publique élus",
    barmode="overlay",
    bargap=0.1,
    xaxis=dict(
        tickvals=[-5000, -2500, 0, 2500, 5000],
        ticktext=["5 000", "2 500", "0", "2 500", "5 000"],
        title="Nombre d’élus"
    ),
    yaxis=dict(title=""),
    legend=dict(orientation="h", y=-0.15),
    margin=dict(l=120, r=20, t=40, b=20),
    font=dict(family="Arial, Helvetica, sans-serif", size=11),
)
dataviz

---
*Made with ❤️ and with [duckit.fr](https://duckit.fr) - [Ali Hmaou](https://www.linkedin.com/in/ali-hmaou-6b7b73146/)*